In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from libs.signals import signals

In [3]:
signals.technical_indicator_signal('btc').tail()

,ewma_x,macd,bollinger,rsi,psar,vwap,signal
Date,,,,,,,
2021-01-02 18:00:00,1,1.0,-1,-1.0,1,1,2.0
2021-01-03 18:00:00,1,1.0,-1,-1.0,-1,0,-1.0
2021-01-04 18:00:00,1,1.0,-1,-1.0,-1,0,-1.0
2021-01-05 18:00:00,1,1.0,-1,-1.0,1,0,1.0
2021-01-06 18:00:00,1,1.0,-1,-1.0,1,0,1.0
